In [16]:
%pip install -q langchain-elasticsearch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
%pip install -q sentence-transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from uuid import uuid4
from elasticsearch import Elasticsearch
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_elasticsearch import ElasticsearchStore
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.retrievers import ElasticSearchBM25Retriever

**Загрузка базы знаний:**

In [2]:
with open(
    file=r"C:\Users\andre\AdvancedRAG\static\texts\ТИУ База знаний.txt",
    mode="r",
    encoding="utf-8"
) as file:
    text = file.read()

print(f"Длина текста: {len(text)}")

Длина текста: 71987


**Разделение текста на чанки:**

In [3]:
'''text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)'''

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=20,
    length_function=len,
)

chunks = text_splitter.create_documents([text])

N = 5

print(f"Всего чанков: {len(chunks)}")
print(f"Первые {N} чанков:")
chunks[:N]

Всего чанков: 173
Первые 5 чанков:


[Document(page_content='Основные сведения:\nПолное наименование образовательной организации\nФедеральное государственное бюджетное образовательное учреждение высшего образования «Тюменский индустриальный университет»\nНаименование организации на иностранном языке\nFederal State Budget Educational Institution of Higher Education «Industrial University of Tyumen»/ IUT\nСокращенное наименование образовательной организации (при наличии)\nФГБОУ ВО «Тюменский индустриальный университет», ФГБОУ ВО «ТИУ», Тюменский индустриальный университет, ТИУ\nДата создания образовательной организации: 08 января 1964 г.'),
 Document(page_content='Краткая информация об образовательной организации:\nТюменский индустриальный университет – опорный вуз региона, лидер в подготовке инженерных кадров.\nИнформация о месте нахождения образовательной организации:\nЮридический адрес: 625000, Уральский федеральный округ, Тюменская область, г. Тюмень, ул. Володарского, 38\nФактический адрес: 625000, Уральский федеральны

**Загрузка Embedding мoдели:**

In [4]:
embeddings = HuggingFaceEmbeddings(
    model_name="ai-forever/sbert_large_nlu_ru",
    model_kwargs={"device": "cpu"},
    encode_kwargs={'normalize_embeddings': False},
)

C:\Users\andre\AppData\Local\Temp\ipykernel_8156\527724830.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(


**Подключение в базе данных (Elastic-search):**

In [5]:
elastic_client = Elasticsearch(
    hosts="https://elasticsearch-pv2s-production.up.railway.app/",
    basic_auth=("elastic", "2qq0lvgc89lwh6z5jp2q9280dcneaf95")
)

In [6]:
indices = elastic_client.cat.indices(h='index').split()

# Удаление всех индексов
for index in indices:
    print(f"Удаляю индекс: {index}")
    elastic_client.indices.delete(index=index, ignore=[400, 404])

print("Все индексы удалены.")

Все индексы удалены.


In [7]:
elastic_store = ElasticsearchStore(
    es_url="https://elasticsearch-pv2s-production.up.railway.app/",
    index_name="tyuiu_index",
    embedding=embeddings,
    es_user="elastic",
    es_password="2qq0lvgc89lwh6z5jp2q9280dcneaf95",
)

**Загрузка документов:**

In [8]:
import os


def read_files_in_directory(directory_path: str) -> str:
    files = []
    if not os.path.exists(directory_path):
        return
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        files.append(file_path)
    return files


directory_path = r'C:\Users\andre\AdvancedRAG\static\chanks'
files = read_files_in_directory(directory_path)

In [9]:
def get_file_name(file_path: str) -> str:
    return file_path.split("\\")[-1]

In [10]:
elastic_store.add_documents(documents=chunks)

['8877035a-077c-4f5d-8a3a-a9929ee1300b',
 'c93509fb-bbc0-436b-a00f-85d7f2528193',
 'a8a4d3f8-0ef6-4c48-a459-46d789c8f257',
 '8b66547f-faec-4032-8f90-83083b5b7e5b',
 '271250a1-484f-4670-8fe2-8c61ac990ace',
 '4750358f-7c62-4c72-b35b-82350b31f6db',
 '750f9688-602c-49f6-b9ed-a1ef8830170e',
 '2bd2e9bc-babe-468d-a38e-20b74fbff81b',
 'faa779eb-823b-46f4-949d-78762922b64a',
 'de74be79-8505-4078-b784-888584661af8',
 'e4c9a805-a6d3-49fd-8843-169c516e8393',
 '17c8680d-c38a-44b3-8570-9c344c74e09e',
 '741f9af8-769b-44bb-95f0-be165254b419',
 '0cf1aa49-c280-4230-a59e-3fa0101a67b6',
 'a4a24dfc-0d40-48ea-aa49-bfae88f89aa7',
 '2255dd6a-1c3a-4b32-82c6-f4ae3bc78942',
 '16abb699-1cc0-4995-9090-2162263480ed',
 'bc569949-dd34-456f-b10b-c2b252b3f8e9',
 '5cf3525c-68be-453f-8396-14c44271d9a8',
 '4c48a4d4-01af-408a-bc99-5a639f667fdd',
 '8f6ade01-7e09-47ed-9706-ec0caca9939e',
 '3f9b9b2f-e450-4696-9f00-ac4944a71ad4',
 '8a1940f9-9e33-4b8d-a89e-a8a1ff33bcd2',
 'cc0cf813-21a6-47d8-9c79-cec6e299fd58',
 'e6d1bc34-3f77-

In [11]:
docs = elastic_store.similarity_search("какой минимальный проходной балл на направление архитектура")
docs

[Document(page_content='В области строительства и транспорта по специальностям «Автомобильные дороги», «Объекты транспортной инфраструктуры», «Производство и применение строительных материалов» и «Организация инвестиционно-строительной деятельности» проходной балл составляет 43 (платно) и 51 (бюджет).'),
 Document(page_content='Курсы ориентированы на подготовку учащихся 10-11 классов к поступлению в ВУЗы по направлениям: 07.03.01 Архитектура, 07.03.03 Дизайн архитектурной среды, 54.03.01 Дизайн.\nПреимущества обучения в Центре архитектурной подготовки:\n- высококвалифицированный педагогический состав, в том числе члены Союза архитекторов, дизайнеров и художников России;\n- программа обучения базируется на специфике архитектурного и дизайнерского творчества;\n- благодаря полученным навыкам выпускники центра поступают в профильные ВУЗы страны и начинают свою деятельность в области архитектуры и дизайна;'),
 Document(page_content='Для направлений, связанных с экономикой и управлением, нап

In [12]:
bm25_retriever = ElasticSearchBM25Retriever(
    client=elastic_client, 
    index_name="docs-tyuiu-index",
    search_kwargs={"k": 2}
)

In [13]:
bm25_retriever.add_texts([document.page_content for document in chunks])

['eb7dd8a4-dbc9-474d-9b9f-fd3f4ded6483',
 'c1546d9d-412f-400b-8d5a-0957bae74e68',
 'fa753036-9a91-4296-bc6d-358361196c13',
 'f6893471-1c62-4e44-9b6d-a3e053ed27e9',
 '0385a067-4b15-47da-b919-87f686f1afc0',
 '2f2a4eb5-a16d-45e3-8991-4efb4b9e264f',
 'ec44d2ef-981d-4709-a7fa-29b79818eafa',
 'db8ed7d0-6541-49f4-abd0-6955aa942925',
 'a77ad7ad-2387-4fb7-bb3c-bfee2e014f3d',
 '4903178d-5939-459e-8abb-02b3b743d63f',
 '31c6ac52-def8-4117-8d1d-91c950233e32',
 '1e19936c-13e2-409c-b6bc-62ead2e76481',
 'cc817a36-62e4-437b-95a0-77e636dedcd6',
 '1849b3c7-4cd8-444e-8957-9d6c61fbedae',
 '288508e2-7aa4-49d4-9932-fae15c61e546',
 '0002fa14-db33-48ff-bdd9-6fa3871650dc',
 'e223dafc-511b-42fb-9f4f-7a6b036a41cd',
 '3ef8895e-b7ef-4a3b-b479-c0cf1cc26767',
 '0a50ee66-6a77-4b38-b407-8258a6f0bfb6',
 'db7fafb4-da1b-4d9b-970d-690be271ae57',
 'cc0c75bd-e65d-4fb1-a10d-67146c4b4411',
 'c0089662-dad7-4792-9ecc-2c0a9cbfcf18',
 '697f2528-3338-4105-b890-b545b76afa19',
 '1d281141-d5c6-4bbc-8195-d0ebf4b8314a',
 '25e36c95-1746-

In [14]:
vector_store_retriever = elastic_store.as_retriever(search_kwargs={"k": 2})

In [15]:
from langchain.retrievers import EnsembleRetriever

In [16]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[vector_store_retriever, bm25_retriever],
    weights=[0.6, 0.4],
)

In [17]:
config = {"configurable": {"search_kwargs_elasticsearch_store": {"k": 1}}}
docs = ensemble_retriever.invoke("Какой проходной балл на архитектуру", config=config)
print(f"Всего документов найдено: {len(docs)}")
for doc in docs:
    print(doc)

Всего документов найдено: 11
page_content='В области строительства и транспорта по специальностям «Автомобильные дороги», «Объекты транспортной инфраструктуры», «Производство и применение строительных материалов» и «Организация инвестиционно-строительной деятельности» проходной балл составляет 43 (платно) и 51 (бюджет).'
page_content='Курсы ориентированы на подготовку учащихся 10-11 классов к поступлению в ВУЗы по направлениям: 07.03.01 Архитектура, 07.03.03 Дизайн архитектурной среды, 54.03.01 Дизайн.
Преимущества обучения в Центре архитектурной подготовки:
- высококвалифицированный педагогический состав, в том числе члены Союза архитекторов, дизайнеров и художников России;
- программа обучения базируется на специфике архитектурного и дизайнерского творчества;
- благодаря полученным навыкам выпускники центра поступают в профильные ВУЗы страны и начинают свою деятельность в области архитектуры и дизайна;'
page_content='Для направлений, связанных с экономикой и управлением, например, «Л